In [ ]:
# !pip install -q youtube-transcript-api \
#  faiss-cpu tiktoken python-dotenv

In [2]:
!pip install faiss

ERROR: Could not find a version that satisfies the requirement faiss (from versions: none)
ERROR: No matching distribution found for faiss


In [ ]:
# !pip install faiss==1.5.3

ERROR: Could not find a version that satisfies the requirement faiss==1.5.3 (from versions: none)
ERROR: No matching distribution found for faiss==1.5.3


In [7]:
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.prompts import PromptTemplate
from youtube_transcript_api import TranscriptsDisabled, YouTubeTranscriptApi
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_community.vectorstores import FAISS
from dotenv import load_dotenv

In [8]:
load_dotenv()

True

In [30]:
ytt_api = YouTubeTranscriptApi()
raw_transcript = ytt_api.get_transcript('ngX3kRdP33E', languages= ['en', 'hi'])

In [31]:
raw_transcript

[{'text': 'Okay, Google just released its agent to',
  'start': 0.08,
  'duration': 5.04},
 {'text': 'aagent protocol, but with its launch,',
  'start': 2.159,
  'duration': 4.961},
 {'text': 'there are some big questions which arise',
  'start': 5.12,
  'duration': 5.04},
 {'text': 'like what about MCP? Like does A2A',
  'start': 7.12,
  'duration': 6.639},
 {'text': 'replace MCP? Is there an overlap? And if',
  'start': 10.16,
  'duration': 6.64},
 {'text': 'not, when do you use what? Well, in this',
  'start': 13.759,
  'duration': 5.121},
 {'text': "stream, we're going to pit A2A against",
  'start': 16.8,
  'duration': 4.639},
 {'text': 'MCP and see what their differences are.',
  'start': 18.88,
  'duration': 6.08},
 {'text': 'Take a good example to highlight some',
  'start': 21.439,
  'duration': 5.76},
 {'text': 'use cases and finally arrive at a',
  'start': 24.96,
  'duration': 4.159},
 {'text': 'framework which helps us decide when to',
  'start': 27.199,
  'duration': 4.80

In [32]:
type(raw_transcript)

list

In [33]:
transcript = " ".join(chunk['text'] for chunk in raw_transcript)

In [34]:
transcript

'Okay, Google just released its agent to aagent protocol, but with its launch, there are some big questions which arise like what about MCP? Like does A2A replace MCP? Is there an overlap? And if not, when do you use what? Well, in this stream, we\'re going to pit A2A against MCP and see what their differences are. Take a good example to highlight some use cases and finally arrive at a framework which helps us decide when to use what. Let\'s get started. I think the best way to compare any frameworks is by looking at an example, right? And we already saw this, so let\'s just go through it again. It can be a quick recap. Okay, let\'s say that I\'m a human. Yeah, a cute little guy. That\'s me. Yep. Right here. And let\'s say I\'m supposed to finish a project by the end of the week, right? And I keep all my project stuff in notion and I keep all my calendar in Google calendar, right? So my projects and my tasks are in notion and everything else is in Google calendar. And I just want to ma

In [35]:
#text splitting
splitter = RecursiveCharacterTextSplitter(chunk_size = 1000, chunk_overlap = 150)
chunks = splitter.create_documents([transcript])

In [36]:
len(chunks)

55

In [38]:
chunks[18]

Document(metadata={}, page_content="I hope this makes sense. There's one more thing I want to highlight. Let's say in an MCP server, we have something called create task and it takes a project and a task. Okay, let's say I have an MCP server which takes two arguments. Now, what if the user's original question was like create this task and they do not mention a project. What would happen? Think about that. What would happen if the user shows intent to call a particular tool but doesn't give you all the inputs? In that case, your client, this guy would have to be smart enough to say that okay, to call this MCP tool, I need some more input. So, I need to ask for more input from the user. Right? The client would have to be smart saying that okay, you know what to call this MCB tool, I need two parameters. The user's question just has one parameter. So, I have to go back to the user and say that hey, you know, user uh you need to give me some more information. So there's a lot of intelligen

In [39]:
#embedding generation and storing in vector store
embedding_model = OpenAIEmbeddings(model= 'text-embedding-3-small')
vector_store = FAISS.from_documents(chunks, embedding_model)

In [40]:
vector_store.index_to_docstore_id

{0: 'aa4214ba-3537-42ce-a33f-2195152a5e94',
 1: '3a4a8d3a-736e-4083-8307-dacceea8d26d',
 2: 'b9e781cd-b15a-4739-8e04-94bc62380c07',
 3: '861d3454-d64f-4ad0-aba9-1133ceb4ddcd',
 4: '070607c6-060f-4cf3-b530-6a0b0560dcb3',
 5: '40ffb8e6-57ef-4e4c-8eb0-4803cfe78f90',
 6: 'f94b7366-638e-4622-a9d4-7c46c1d653c8',
 7: 'c05102b1-42ee-4119-be1f-4899975fe9b2',
 8: '98467f12-944b-4cf8-bd4d-85834ca4f751',
 9: '0f61133a-8d3c-4e15-a0a5-9dc50eb42448',
 10: '7c24570b-daaa-4071-a9f9-f94a09b29eb2',
 11: '46d1de4d-7201-4c99-b669-e63ca64bbb81',
 12: 'd8148c43-b38c-44f5-ad24-b0ab87d6a454',
 13: '028ee637-48d1-4242-8365-9d3cdd7722d7',
 14: '93ea5cfd-addf-46ca-8131-922b23de8a1e',
 15: 'd7152eb6-093d-4a56-9873-750d4c9ca001',
 16: '48fb72ee-3229-4a22-b40e-8cee9121d7d1',
 17: 'f1955954-ad9f-4ec1-a71a-5342edf13130',
 18: '3a06fb33-80a7-4b40-a637-18b9a6745d47',
 19: '71921346-142c-4e59-b037-2a4cb65d4492',
 20: 'c9818f92-5a81-4ef6-9968-5354f8361ebe',
 21: 'aef0c5e8-6fd8-4cf9-95be-e07475d48317',
 22: 'd5dbafb2-e434-

In [41]:
vector_store.get_by_ids(['720a12a5-d371-4a23-b12e-134a8a1c77ad'])

[Document(id='720a12a5-d371-4a23-b12e-134a8a1c77ad', metadata={}, page_content="A28 talk to other agents collaborate. Yes MCP use tools precisely. I think that's the perfect place to even end things, right? So the conclusion would be A2A allows for intelligence on the server. So whenever you want to create a server and you think you're going to use an LLM there, A2A would be a good bet. You can do it with MCP, but eventually you run into this problems like what if you want to collaborate? What if you want to ask user follow-up questions? What if you want to stream? What if you want to have events? MCP currently does not support those things. So if you're having an LLM on it server on an MCP server, you're going to lose out on this features which is a little annoying. Away does that really well, but A2A is a complex protocol. So if you just have a server which does things like these, A2A would be an overkill, especially the fact that it can't even support these things. Let me know how a

In [42]:
#retrieval 
retriever = vector_store.as_retriever(search_type = 'similarity', search_kwargs = {"k": 5})

In [43]:
retriever

VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x000002547FBF3BE0>, search_kwargs={'k': 5})

In [44]:
retriever.invoke("What is MCP")

[Document(id='48fb72ee-3229-4a22-b40e-8cee9121d7d1', metadata={}, page_content="in MCP which says send task and the message could be any. It could be string, it could be binary, it could be anything. And now you have the same kind of API for both of them, right? So A2A is kind of like MCP with a single tool. That's what I'm trying to highlight. A2A is like MCP server with a single tool where you just take a message as the input and the message could be anything. It could be natural language string. It could be structured. It could be a JSON object. It could literally be anything. However, most people will use MCP more like this like get tasks uh create task and stuff like that, right? Create task and maybe you have a task as well, whatever. You get the point, guys. Again, A2A is like an MCP server with just a single tool. All right? However, in MCP, the spec currently at least currently only has a description. So, the only way you can describe what this tool does is through a descripti

In [54]:
#Augmentation
llm = ChatOpenAI(temperature= 0.25)


In [45]:
prompt = PromptTemplate(template = """You are a helpful assistant.
      Answer ONLY from the provided transcript context.
      If the context is insufficient, just say you don't know.

      {context}
      Question: {question}""",
      input_variables= ['context', 'question'])

In [47]:
question = "Is the topic of AI Agent discussed in this video?"
retrieved_docs = retriever.invoke(question)

In [48]:
retrieved_docs

[Document(id='aa4214ba-3537-42ce-a33f-2195152a5e94', metadata={}, page_content="Okay, Google just released its agent to aagent protocol, but with its launch, there are some big questions which arise like what about MCP? Like does A2A replace MCP? Is there an overlap? And if not, when do you use what? Well, in this stream, we're going to pit A2A against MCP and see what their differences are. Take a good example to highlight some use cases and finally arrive at a framework which helps us decide when to use what. Let's get started. I think the best way to compare any frameworks is by looking at an example, right? And we already saw this, so let's just go through it again. It can be a quick recap. Okay, let's say that I'm a human. Yeah, a cute little guy. That's me. Yep. Right here. And let's say I'm supposed to finish a project by the end of the week, right? And I keep all my project stuff in notion and I keep all my calendar in Google calendar, right? So my projects and my tasks are in 

In [50]:
context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
context_text

"Okay, Google just released its agent to aagent protocol, but with its launch, there are some big questions which arise like what about MCP? Like does A2A replace MCP? Is there an overlap? And if not, when do you use what? Well, in this stream, we're going to pit A2A against MCP and see what their differences are. Take a good example to highlight some use cases and finally arrive at a framework which helps us decide when to use what. Let's get started. I think the best way to compare any frameworks is by looking at an example, right? And we already saw this, so let's just go through it again. It can be a quick recap. Okay, let's say that I'm a human. Yeah, a cute little guy. That's me. Yep. Right here. And let's say I'm supposed to finish a project by the end of the week, right? And I keep all my project stuff in notion and I keep all my calendar in Google calendar, right? So my projects and my tasks are in notion and everything else is in Google calendar. And I just want to make sure\

In [51]:
final_prompt = prompt.invoke({"context": context_text, "question": question})

In [52]:
final_prompt

StringPromptValue(text="You are a helpful assistant.\n      Answer ONLY from the provided transcript context.\n      If the context is insufficient, just say you don't know.\n\n      Okay, Google just released its agent to aagent protocol, but with its launch, there are some big questions which arise like what about MCP? Like does A2A replace MCP? Is there an overlap? And if not, when do you use what? Well, in this stream, we're going to pit A2A against MCP and see what their differences are. Take a good example to highlight some use cases and finally arrive at a framework which helps us decide when to use what. Let's get started. I think the best way to compare any frameworks is by looking at an example, right? And we already saw this, so let's just go through it again. It can be a quick recap. Okay, let's say that I'm a human. Yeah, a cute little guy. That's me. Yep. Right here. And let's say I'm supposed to finish a project by the end of the week, right? And I keep all my project st

In [55]:
answer = llm.invoke(final_prompt)
print(answer.content)

Yes, the topic of AI Agent is discussed in this video, specifically comparing A2A and MCP frameworks.


In [56]:
from langchain_core.runnables import RunnableLambda, RunnablePassthrough, RunnableParallel
from langchain_core.output_parsers import StrOutputParser


In [57]:
def format_docs(retrieved_docs):
    context_text = "\n\n".join(doc.page_content for doc in retrieved_docs)
    return context_text

In [58]:
parallel_chain = RunnableParallel({
    'context': retriever | RunnableLambda(format_docs),
    'question': RunnablePassthrough()
})


In [59]:
parallel_chain.invoke('What is AI Agent')

{'context': "an artifact called order confirmation and you can have some details. If you were generating an image, the artifact would be the image. If you were creating a task, the task ID could be the artifact. The point is once you get the artifact, you know for sure that hey, things are done. I have an answer. Now I can be dumb and just show this to the user or I can use this to I make a decision, call another A2A agent or whatever. You see how explicit A2A is. The agent has first class capability to request for additional input. And we saw that in our example as well when we were trying to schedule some tasks with the calendar. Essentially what was happening is the calendar did not have a 4hour block free. So it asks a question. It sends back a message saying hey I need some more questions. Again the user could just you know take that and handle it directly. But in our case our assistance was intelligent. Again there's nothing wrong with intelligence right? the the assistance was i

In [60]:
parser = StrOutputParser()

In [62]:
main_chain = parallel_chain | prompt | llm | parser

In [63]:
main_chain.invoke('what is an AI agent')

'An AI agent is a software program that can perform tasks, make decisions, and interact with users using natural language. It can request additional input, handle incomplete information, and provide intelligent responses. In the context of A2A (Agent to Agent) communication, the intelligence of the agent is primarily on the server side, but in A2A, intelligence can also be on the client side. A2A agents have broad capabilities and can handle various queries in natural language.'

In [64]:
main_chain.invoke('Summary of the video in 5 points')

"1. A2A focuses on proving why a conclusion is better rather than just arriving at a conclusion.\n2. The scope of A2A is not well defined due to the broad skills it possesses.\n3. A2A supports streaming responses, push notifications, and task state updates.\n4. A2A has skills like route planning, which are described with examples for clarity.\n5. A2A's capability discovery is more organized and concise compared to MCP's tool descriptions."